# Spectral analysis using QGSP

This example demonstrates how to build a quaternion graph signal, a quaternion-weighted graph and how to perform a spectral analysis of such signal.

In [ ]:
# If gspx is not installed, we add it to the path
import os, sys
gdir = os.path.dirname(os.getcwd())  # parent folder
sys.path.insert(0, gdir)

In [ ]:
import numpy as np

from gspx.utils.display import plot_graph
from gspx.datasets import SocialGraphData
from gspx.signals import QuaternionSignal
from gspx.qgsp import create_quaternion_weights, QGFT, QMatrix

## Quaternion-weighted graph

In [ ]:
data = SocialGraphData()
Ar, coords = data.graph
s = data.signal

In [ ]:
df = data.data

Aq = create_quaternion_weights(
    Ar, df, icols=['median_household_income_2017'],
    jcols=['unemployment_rate_2017'],
    kcols=['uninsured_2017'], gauss_den=0.5)

In [ ]:
plot_graph(
    Aq.abs(), coords=coords, colors=s.to_array()[1:, 0],
    figsize=(10, 5), colormap='viridis',
    node_size=20)

In [ ]:
qgft = QGFT()
qgft.fit(Aq)

## Total variation of each eigenvector for each eigenvalue

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.scatter(np.real(qgft.eigc), np.imag(qgft.eigc), c=qgft.tv_)
plt.colorbar()
plt.title("Total Variation of eigenvectors for each eigenvalue")
plt.xlabel("Real(eigvals)")
plt.ylabel("Imag(eigvals)")
plt.savefig('../../phd-thesis/thesis/Figures/uk_qgsp_tv.pdf', dpi=300)
plt.show()

In [ ]:
obj = QuaternionSignal.from_samples(s.matrix.ravel())
node_color = [tuple(rgba) for rgba in obj.to_rgba()]

plot_graph(
    Aq.abs(), coords=coords, colors=node_color,
    figsize=(4, 8), colormap='viridis',
    node_size=40)
# plt.savefig('../../phd-thesis/thesis/Figures/uk_qgsp_graphsig.pdf', dpi=300)

In [ ]:
ss = qgft.transform(s)

In [ ]:
fig = QuaternionSignal.show(ss, ordering=qgft.idx_freq)
fig.savefig('../../phd-thesis/thesis/Figures/uk_qgsp_spectrumsig.pdf', dpi=300)